# Data Setup For Maps

By: Adam Li

Following tutorials on D3.js, I want to create maps of different cities in the USA for data visualization. To get the geodata of any city, I am using the US package: https://pypi.python.org/pypi/us, which provides detailed information about:
* all US states and territories
* postal abbreviations
* Associated Press style abbreviations
* FIPS codes
* capitals
* years of statehood
* time zones
* phonetic state name lookup
* is contiguous or continental
* URLs to shapefiles for state, census, congressional districts, counties, and census tracts

## Methods:
1. GET SHAPE GEO DATA: First we gather shape data from the TIGER site. This requires us to initialize the full tiger url, state code, county lists. Then we unzip the file and store it locally. Then we gather water data from the TIGER site. 
2. CREATE GEO DATA JSON: With the shape and water data, we create a geo_data.json file that can reproduce the geo map of the location we are interested in.
3. CENSUS TRACT information: to populate the geojson areas with some sort of data
4. CREATE CENSUS DATA CSV: using dataframes, convert to csv easily

### References:
- Census API: https://github.com/sunlightlabs/census
- FIPS Explanation: https://www.policymap.com/blog/2012/08/tips-on-fips-a-quick-guide-to-geographic-place-codes-part-iii/

In [57]:
import os.path
from census import Census
from us import states
import geopandas as gpd
import pandas as pd
import zipfile
import requests

from IPython.display import display

### MAIN FUNCTIONS: 
- build the fips code for a blockgroup
- build fips code for tracts
- converting census blockgroup to a panda dataframe
- check if a directory exists and if not create it
- convert census blockgroup to dataframe
- convert census tract to dataframe

In [2]:
def build_bg_fips(record):
    fips_code = record['state'] + record['county'] + record['tract'] + record['block group']
    return str(fips_code)

def build_tract_fips(record):
    fips_code = record['state'] + record['county'] + record['tract']
    return str(fips_code)

def census_to_dataframe(var_list, state_code, county_codes):
    fips_codes = []
    all_records = []
    
    for county in county_codes:        
        census_data = c.acs.get(var_list, {'for': 'block group:*', 'in': 'state:{0} county:{1}'.format(state_code, county)})
        
        for idx, record in enumerate(census_data):
            # Build fips codes
            fips_code = build_bg_fips(record)
            census_data[idx]["fips"] = fips_code

            # Eliminate original code components
            key_list = ['state', 'county', 'tract', 'block group']
            for key in key_list:
                if key in census_data[idx]: 
                    del census_data[idx][key]
        
        all_records.extend(census_data)
        
    census_df = pd.DataFrame(all_records)
    census_df = census_df.set_index("fips")
                
    return census_df

def checkDirExist(filepath):
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        
def census_bg_to_dataframe(var_list, state_code, county_codes):
    fips_codes = []
    all_records = []
    
    for county in county_codes:        
        census_data = c.acs.get(var_list, {'for': 'tract:*', 'in': 'state:{0}'.format(state_code)})
        
        for idx, record in enumerate(census_data):
            # Build fips codes
            fips_code = build_bg_fips(record)
            census_data[idx]["fips"] = fips_code

            # Eliminate original code components
            key_list = ['state', 'county', 'tract', 'block group']
            for key in key_list:
                if key in census_data[idx]: 
                    del census_data[idx][key]
        
        all_records.extend(census_data)
        
    census_df = pd.DataFrame(all_records)
    census_df = census_df.set_index("fips")
                
    return census_df

def census_tracts_to_dataframe(var_list, state_codes):
    fips_codes = []
    all_records = []
    
    for state_id in state_codes:
        census_data = c.acs.get(var_list, {'for': 'tract:*', 'in': 'state:{0}'.format(state_id)})

        for idx, record in enumerate(census_data):

            # Build fips codes
            fips_code = build_tract_fips(record)
            census_data[idx]["fips"] = fips_code

            # Eliminate original code components
            key_list = ['state', 'county', 'tract']
            for key in key_list:
                if key in census_data[idx]: 
                    del census_data[idx][key]
        
        all_records.extend(census_data)
      
    census_df = pd.DataFrame(all_records)
    census_df = census_df.set_index("fips")
                
    return census_df

### Initialize CENSUS API and URLs For Online GEO Files

In [3]:
# CENSUS API Stuff
CENSUS_API = 'fe55211c8b3f0350fcb040c07321a129a3d6e266' # Your key here
c = Census(CENSUS_API) # Initialize census class with API key

# TIGER Stuff
TIGER_BASE_URL = 'http://www2.census.gov/geo/tiger/TIGER2013/'
TIGER_TRACT_DIR = 'TRACT/'
TIGER_BLOCKGROUP_DIR = 'BG/'
TIGER_WATER_DIR = 'AREAWATER/'

# LOCAL DATA DIR FOR STORING THE DATA FROM CENSUS.GOV
LOCAL_DATA_DIR = './data/'
GEO_SUB_DIR = 'geo/'

# FILE ENDINGS FOR FINAL DATA TO BE SAVED
ATTR_FILE_END = '_census_data.csv'
GEO_FILE_END = '_geo_data.json'

# Parameter Settings for Which states, counties

In [33]:
# Specify state and county to download (select one)
loc_name, state_codes, county_codes = "balt_city", [states.MD.fips], list([510]) # Baltimore
# Specify state and county to download (select one)
loc_name, state_codes, county_codes = 'maryland', states.MD.fips, None
# loc_name, state_codes, county_codes = 'delmarva', [states.MD.fips, states.DE.fips, states.VA.fips], None
# loc_name, state_codes, county_codes = 'california', states.CA.fips, None
loc_name, state_code, county_codes = "balt_city", states.MD.fips, list([510]) # Baltimore

# Create county list (string representation of county IDs)
county_list = ["{:03d}".format(county_id) for county_id in county_codes]

# Generate codes for census variables of interest
var_ids = ["B19001_0{:02d}E".format(x) for x in range(2, 18)] # Household income over 12 months

# GET FILE BY STATE_CODE
tiger_shape_file = 'tl_2013_{0}_bg.shp'.format(state_code)

attr_outfile = LOCAL_DATA_DIR + loc_name + ATTR_FILE_END
geo_outfile = LOCAL_DATA_DIR + loc_name + GEO_FILE_END

In [34]:
print "Maryland state code is: ", state_code
print county_list
print var_ids
print tiger_zip_file
print tiger_shape_file
print loc_name

Maryland state code is:  24
['510']
['B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E', 'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E', 'B19001_012E', 'B19001_013E', 'B19001_014E', 'B19001_015E', 'B19001_016E', 'B19001_017E']
tl_2013_2_bg.zip
tl_2013_24_bg.shp
balt_city


# 1. Get TIGER data (shape data) From TIGER

### get data by state at the blockgroup level

In [42]:
for state_id in state_codes:
    tiger_zip_file = 'tl_2013_{0}_bg.zip'.format(state_id)

    FULL_TIGER_URL = TIGER_BASE_URL + TIGER_BLOCKGROUP_DIR + tiger_zip_file
    print "On state_code: ", state_id
    print FULL_TIGER_URL

    # Check if file is in directory, else download it
    if os.path.isfile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file):
        print "Already had the file.  Great.", tiger_zip_file
    else:
        r = requests.get(FULL_TIGER_URL)

        if r.status_code == requests.codes.ok:
            print "Got files, copying to disk..."
            checkDirExist(LOCAL_DATA_DIR + GEO_SUB_DIR)
            with open(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file, 'wb') as f:
                f.write(r.content)
        else:
            print "Error with getting the data. Status code: ".format(r.status_code)

        # Unzip file, extract contents
        zfile = zipfile.ZipFile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file)
        zfile.extractall(LOCAL_DATA_DIR + GEO_SUB_DIR)  

On state_code:  24
http://www2.census.gov/geo/tiger/TIGER2013/BG/tl_2013_24_bg.zip
Got files, copying to disk...


In [70]:
state_shapes = [] # list of geodataframes
for idx, state_id in enumerate(state_codes):
    tiger_shape_file = 'tl_2013_{0}_tract.shp'.format(state_id)
    print "Loading GeoPD from: ", tiger_shape_file

    # Load to GeoDataFrame
    state_shape = gpd.GeoDataFrame.from_file(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_shape_file)
    # Only keep counties that we are interested in
    if county_list is not None:
        print "Only keeping counties in: ", county_list
        state_shape = state_shape[state_shape["COUNTYFP"].isin(county_list)]
    
    state_shapes.append(state_shape)
       
# concatenate the dataframes from the tract level
shapes = gpd.GeoDataFrame( pd.concat(state_shapes, ignore_index=True) )

print "The shape of the new dataframe is: ", shapes.shape

Loading GeoPD from:  tl_2013_24_tract.shp
The shape of the new dataframe is:  (200, 13)


### GET TIGER (water data) from Site

In [45]:
# Check if file is in directory, else download it
for county in county_list:
    tiger_water_zip_file = "tl_2013_{0}{1}_areawater.zip".format(state_code, county)

    if os.path.isfile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_water_zip_file):
        print "Already had the file.  Great.", tiger_water_zip_file
    else:
        r = requests.get(TIGER_BASE_URL + TIGER_WATER_DIR + tiger_water_zip_file)

        if r.status_code == requests.codes.ok:
            print("Got the file! Copying to disk.")
            with open(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_water_zip_file, "wb") as f:
                f.write(r.content)
        else:
            print("Something went wrong. Status code: ".format(r.status_code))
    
    # Unzip file, extract contents
    zfile = zipfile.ZipFile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_water_zip_file)
    zfile.extractall(LOCAL_DATA_DIR + GEO_SUB_DIR)

Got the file! Copying to disk.


## 2. WRITE SHAPE DATA TO GEOJSON

In [46]:
small_shapes = gpd.GeoDataFrame()
small_shapes["geometry"] = shapes["geometry"].simplify(tolerance=0.0001) # Simplify geometry to reduce file size
small_shapes["fips"] = shapes["GEOID"]
small_shapes = small_shapes.set_index("fips") # set index to the geo IDs of polygons

small_json = small_shapes.to_json()

# Write to file
print "Writing to ", geo_outfile
with open(geo_outfile, 'w') as f:
    f.write(small_json)

Writing to  ./data/balt_city_geo_data.json


## 3. GET CENSUS DATA

In [71]:
# get census tracts for the corresponding state
for state_id in state_codes:
    census_data = c.acs.get(var_ids, {'for': 'tract:*', 'in': 'state:{0}'.format(state_id)})
    census_df = pd.DataFrame(census_data)
    print state_id
    display(census_df.head())

24


,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,county,state,tract
0,132,84,80,98,93,83,80,99,68,215,165,139,134,17,14,0,001,24,000100
1,69,92,78,43,80,80,75,61,24,108,127,141,99,30,65,17,001,24,000200
2,51,164,29,109,42,165,80,65,10,87,92,113,60,79,0,8,001,24,000300
3,87,70,52,32,104,115,120,39,82,77,59,51,149,51,8,45,001,24,000400
4,354,134,72,53,32,50,59,14,38,97,35,27,26,32,13,0,001,24,000500


# 4. CREATE CSV CENSUS FILE

In [72]:
# This segment of code will get household income estimates for each block group in state_code
census_df = census_tracts_to_dataframe(var_ids, state_codes)
census_df.to_csv(attr_outfile)

print "Converting census dataframe to csv file."

Converting census dataframe to csv file.


## OPTIONAL: CENSUS Tract Information LEVEL

Use this code to get information at the tract level

In [59]:
for state_id in state_codes:
    tiger_zip_file = 'tl_2013_{0}_tract.zip'.format(state_id)
    
    FULL_TIGER_URL = TIGER_BASE_URL + TIGER_TRACT_DIR + tiger_zip_file
    print "On state_code: ", state_id
    print FULL_TIGER_URL
    
    # Check if file is in directory, else download it
    if os.path.isfile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file):
        print "Already had the file.  Great.", tiger_zip_file
    else:
        r = requests.get(FULL_TIGER_URL)

        if r.status_code == requests.codes.ok:
            print("Got the file! Copying to disk.")
            with open(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file, "wb") as f:
                f.write(r.content)
        else:
            print("Something went wrong. Status code: ".format(r.status_code))
            
    # Unzip file, extract contents
    zfile = zipfile.ZipFile(LOCAL_DATA_DIR + GEO_SUB_DIR + tiger_zip_file)
    zfile.extractall(LOCAL_DATA_DIR + GEO_SUB_DIR)

On state_code:  24
http://www2.census.gov/geo/tiger/TIGER2013/TRACT/tl_2013_24_tract.zip
Already had the file.  Great. tl_2013_24_tract.zip
